In [ ]:
# Import Packages
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import sys
import os
import pickle
# Add the src/ folder to Python path
sys.path.append(os.path.abspath("../src"))
from models import *

2025-12-12 20:33:30.182921: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-12-12 20:33:30.182953: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-12-12 20:33:30.182958: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2025-12-12 20:33:30.182977: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-12 20:33:30.182991: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Data Preparation and Preprocessing

In [ ]:
root_path = "../data/processed"

# Set directory paths for train/val/test splits
train_dir = os.path.join(root_path, "train")
val_dir   = os.path.join(root_path, "val")
test_dir  = os.path.join(root_path, "test")

# Image and loading parameters
img_size = (224, 224)
batch_size = 32
seed = 42

In [5]:
# Load training images with labels, shuffling enabled
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,            # resize images to model input size
    batch_size=batch_size,
    label_mode="categorical",       # one-hot encoded labels
    shuffle=True,
    seed=seed,                      # ensures reproducible shuffling
)

# Load validation images (no shuffling)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

# Load test images (no shuffling)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical",
    shuffle=False,
)

# Extract class names and count number of classes
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


Found 4426 files belonging to 5 classes.
Found 948 files belonging to 5 classes.
Found 949 files belonging to 5 classes.
Classes: ['F0', 'F1', 'F2', 'F3', 'F4']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# Cache data, shuffle training set, and prefetch for faster I/O
train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)

# Cache and prefetch validation and test sets (no shuffle)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

In [ ]:
y_int = []

# Convert one-hot labels - integer class indices
for _, labels in train_ds.unbatch():
    y_int.append(tf.argmax(labels).numpy())

y_int = np.array(y_int)

# Compute balanced class weights
weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(num_classes),
    y=y_int
)

# Map class index → weight
class_weights = {i: w for i, w in enumerate(weights)}
print("Class weights:", class_weights)

Class weights: {0: 0.5981081081081081, 1: 1.4679933665008291, 2: 1.5949549549549549, 3: 1.4753333333333334, 4: 0.7451178451178451}


2025-12-11 12:08:25.572115: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Model Training

### Baseline CNN

In [ ]:
# Build and compile baseline CNN model
# Uses Adam optimizer and categorical cross-entropy loss
# Tracks accuracy, precision, and recall during training
baseline_model = build_baseline_cnn(num_classes=num_classes)

baseline_model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

baseline_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,453 (501.77 KB)

 Trainable params: 128,005 (500.02 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Train the baseline CNN
# Uses class weights to address class imbalance
# Evaluates on validation set each epoch
history_baseline = baseline_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    class_weight=class_weights,
)

Epoch 1/30


2025-12-11 12:08:26.811317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


139/139 ━━━━━━━━━━━━━━━━━━━━ 109s 760ms/step - accuracy: 0.3371 - loss: 1.5416 - precision: 0.3995 - recall: 0.1405 - val_accuracy: 0.1255 - val_loss: 1.6615 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 127s 909ms/step - accuracy: 0.3900 - loss: 1.3806 - precision: 0.4546 - recall: 0.2126 - val_accuracy: 0.3376 - val_loss: 1.6802 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 82s 591ms/step - accuracy: 0.3940 - loss: 1.3613 - precision: 0.4578 - recall: 0.2307 - val_accuracy: 0.4019 - val_loss: 1.7376 - val_precision: 0.3478 - val_recall: 0.0084
Epoch 4/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 64s 460ms/step - accuracy: 0.3958 - loss: 1.3478 - precision: 0.4554 - recall: 0.2366 - val_accuracy: 0.4884 - val_loss: 1.4369 - val_precision: 0.7122 - val_recall: 0.2532
Epoch 5/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 61s 439ms/step - accuracy: 0.4038 - loss: 1.3530 - precision: 0.4636 - recall: 0.2445 - val_accuracy:

In [ ]:
# Evaluate the trained baseline model on the held-out test set
print("Testing on external test dataset…")
baseline_model.evaluate(test_ds)

Testing on external test dataset…
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.4552 - loss: 2.0694 - precision: 0.4812 - recall: 0.4447


[2.0694210529327393,
 0.45521602034568787,
 0.4811858534812927,
 0.4446786046028137]

### Resnet50

In [ ]:
# Build and compile the ResNet50 transfer learning model
resnet_model, resnet_base = build_resnet50(num_classes=num_classes)

resnet_model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

resnet_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_augmentation   │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ data_augmentatio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ data_augmentatio… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ data_augmentatio… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 2048)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 512)       │  1,049,088 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 512)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5)         │      2,565 │ dropout_5[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train ResNet50 with frozen base (feature extractor mode)
history_res_frozen = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights,
)


Epoch 1/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 83s 575ms/step - accuracy: 0.3534 - loss: 2.3305 - precision_1: 0.3619 - recall_1: 0.3143 - val_accuracy: 0.4821 - val_loss: 1.2126 - val_precision_1: 0.5275 - val_recall_1: 0.4546
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 77s 553ms/step - accuracy: 0.4160 - loss: 2.0000 - precision_1: 0.4282 - recall_1: 0.3784 - val_accuracy: 0.5854 - val_loss: 0.9776 - val_precision_1: 0.6615 - val_recall_1: 0.4989
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 849s 6s/step - accuracy: 0.4659 - loss: 1.9108 - precision_1: 0.4853 - recall_1: 0.4358 - val_accuracy: 0.6150 - val_loss: 0.8947 - val_precision_1: 0.7019 - val_recall_1: 0.5464
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 47s 339ms/step - accuracy: 0.4756 - loss: 1.8763 - precision_1: 0.4927 - recall_1: 0.4446 - val_accuracy: 0.6297 - val_loss: 0.8572 - val_precision_1: 0.7102 - val_recall_1: 0.5559
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 51s 368ms/step - accuracy: 0.4826 - loss: 1.8418 - precision_1: 0.4991 - recal

In [ ]:
# Unfreeze top layers of ResNet50 for fine-tuning
resnet_base.trainable = True

for layer in resnet_base.layers[:-30]:
    layer.trainable = False

# Re-compile with lower LR for stable fine-tuning
resnet_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

# Fine-tune the model
history_res_ft = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights,
)

Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 102s 693ms/step - accuracy: 0.5585 - loss: 1.4761 - precision_2: 0.5859 - recall_2: 0.5258 - val_accuracy: 0.7068 - val_loss: 0.7667 - val_precision_2: 0.7529 - val_recall_2: 0.6783
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 93s 671ms/step - accuracy: 0.6263 - loss: 1.2235 - precision_2: 0.6586 - recall_2: 0.5949 - val_accuracy: 0.7489 - val_loss: 0.6364 - val_precision_2: 0.8012 - val_recall_2: 0.6930
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 94s 677ms/step - accuracy: 0.6663 - loss: 1.1072 - precision_2: 0.7017 - recall_2: 0.6403 - val_accuracy: 0.7753 - val_loss: 0.5403 - val_precision_2: 0.8451 - val_recall_2: 0.7194
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 94s 677ms/step - accuracy: 0.7088 - loss: 0.9957 - precision_2: 0.7424 - recall_2: 0.6778 - val_accuracy: 0.7795 - val_loss: 0.4930 - val_precision_2: 0.8514 - val_recall_2: 0.7194
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 98s 706ms/step - accuracy: 0.7164 - loss: 0.9300 - precision_2: 0.7548 - re

In [ ]:
# Evaluate fine-tuned ResNet50 on the held-out test set
resnet_model.evaluate(test_ds)

30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 245ms/step - accuracy: 0.9083 - loss: 0.2936 - precision_2: 0.9157 - recall_2: 0.9041


[0.293574720621109, 0.9083245396614075, 0.9156883955001831, 0.9041095972061157]

### EfficientNetB0

In [ ]:
# Build EfficientNetB0 feature extractor + classifier head
eff_model, eff_base = build_effnet(num_classes=num_classes)

# Compile model with optimizer, loss, and evaluation metrics
eff_model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

# Print model architecture
eff_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,378,792 (16.70 MB)

 Trainable params: 329,221 (1.26 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# Train EfficientNetB0 with frozen base (feature extractor)
history_eff_frozen = eff_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights,
)


Epoch 1/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 40s 248ms/step - accuracy: 0.3970 - loss: 1.5138 - precision_3: 0.4732 - recall_3: 0.2357 - val_accuracy: 0.5390 - val_loss: 1.0192 - val_precision_3: 0.7756 - val_recall_3: 0.3281
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 37s 270ms/step - accuracy: 0.4645 - loss: 1.3353 - precision_3: 0.5491 - recall_3: 0.3477 - val_accuracy: 0.5949 - val_loss: 0.9506 - val_precision_3: 0.8355 - val_recall_3: 0.3428
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 49s 349ms/step - accuracy: 0.4767 - loss: 1.2805 - precision_3: 0.5617 - recall_3: 0.3671 - val_accuracy: 0.5907 - val_loss: 0.9160 - val_precision_3: 0.8028 - val_recall_3: 0.3650
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 43s 307ms/step - accuracy: 0.5002 - loss: 1.2507 - precision_3: 0.5784 - recall_3: 0.3900 - val_accuracy: 0.6361 - val_loss: 0.8639 - val_precision_3: 0.8555 - val_recall_3: 0.3935
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 43s 310ms/step - accuracy: 0.5217 - loss: 1.2097 - precision_3: 0.6058 - rec

In [ ]:
# Unfreeze top layers of EfficientNetB0 for fine-tuning
eff_base.trainable = True
for layer in eff_base.layers[:-40]:
    layer.trainable = False  # Freeze lower layers to retain pretrained features

# Compile model with lower learning rate for fine-tuning
eff_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()],
)

# Train the model with fine-tuning
history_eff_ft = eff_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weights,
)

Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 80s 517ms/step - accuracy: 0.5556 - loss: 1.1972 - precision_4: 0.6391 - recall_4: 0.4494 - val_accuracy: 0.6677 - val_loss: 0.7855 - val_precision_4: 0.7726 - val_recall_4: 0.5591
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 70s 501ms/step - accuracy: 0.5798 - loss: 1.1062 - precision_4: 0.6820 - recall_4: 0.4521 - val_accuracy: 0.6888 - val_loss: 0.7674 - val_precision_4: 0.8052 - val_recall_4: 0.5580
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 67s 483ms/step - accuracy: 0.5917 - loss: 1.0656 - precision_4: 0.7112 - recall_4: 0.4690 - val_accuracy: 0.6825 - val_loss: 0.7533 - val_precision_4: 0.8173 - val_recall_4: 0.5475
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 68s 488ms/step - accuracy: 0.6078 - loss: 1.0470 - precision_4: 0.7082 - recall_4: 0.4772 - val_accuracy: 0.6994 - val_loss: 0.7367 - val_precision_4: 0.8265 - val_recall_4: 0.5527
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 70s 503ms/step - accuracy: 0.6263 - loss: 1.0150 - precision_4: 0.7270 - rec

In [ ]:
# Evaluate the fine-tuned EfficientNetB0 model on the held-out test data
eff_model.evaluate(test_ds)

30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 292ms/step - accuracy: 0.7534 - loss: 0.5792 - precision_4: 0.8754 - recall_4: 0.6586


[0.5791510939598083,
 0.7534246444702148,
 0.8753501176834106,
 0.6585879921913147]

# Save Trained Models

In [ ]:
# Create a directory to save trained models (if it doesn't exist)
os.makedirs("saved_models", exist_ok=True)

# Save the trained models in Keras format for later use or deployment
baseline_model.save("saved_models/baseline.keras")
resnet_model.save("saved_models/resnet50.keras")
eff_model.save("saved_models/effnetB0.keras")

In [ ]:
# Save the training histories using pickle for future analysis or plotting
with open("saved_models/history_baseline.pkl", "wb") as f:
    pickle.dump(history_baseline.history, f)

with open("saved_models/history_resnet.pkl", "wb") as f:
    pickle.dump(history_res_ft.history, f)

with open("saved_models/history_effnet.pkl", "wb") as f:
    pickle.dump(history_eff_ft.history, f)